<a href="https://colab.research.google.com/github/govind17/Information-Retrieval-Project/blob/main/LDA_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U Flask
# !pip install git+https://github.com/boudinfl/pke.git
# !pip install matplotlib
!python -m spacy download en_core_web_sm

In [ ]:

!pip install flask-script


In [1]:
# One time installation
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/GovindShukla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/GovindShukla/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import numpy as np
import pandas as pd
import operator
import io
# import flask_script
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import spacy

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize

import flask
from flask import request
from flask_cors import CORS
from flask import Flask, Response
from keyPhrasification import key_phrasification

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from matplotlib.figure import Figure

from io import BytesIO

import os
import joblib
import pyterrier as pt
from pyterrier_t5 import MonoT5ReRanker
import pandas as pd
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import ast
from sklearn.manifold import TSNE
from keyPhrasification import key_phrasification
# from flask_script import Manager


In [3]:
# Indexing the cord-19 dataset
if not pt.started():
    pt.init()

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = './terrier_cord19'
if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer
    indexer = pt.index.IterDictIndexer(pt_index_path)
    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(),
                              fields=('abstract',),
                              meta=('docno',))
else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")
    index = pt.IndexFactory.of(index_ref)


PyTerrier 0.9.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [4]:
#text preprocessing
def pre_processing_data(data):
    ps = PorterStemmer()
    tagged_dataset = [TaggedDocument(words=[ps.stem(w) for w in nltk.word_tokenize(str(_d)) if word_tokenize(str(_d).lower()) not in stopwords.words('english')], tags=[str(i)]) for i, _d in enumerate(data)]
    #tagged_dataset = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
    return tagged_dataset

def model_doc2vec(model, tagged_data, num_epochs):
    model.build_vocab(tagged_data)
    model.train(tagged_data,total_examples=len(tagged_data), epochs=num_epochs)
    return model

def vector_for_learning(model, input_docs):
    sents = input_docs
    targets, feature_vectors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, feature_vectors

def define_d2v_model(dataset):
    preprocessed_tagged_dataset = pre_processing_data(dataset)
    model_d2v = Doc2Vec(dm=1, vector_size=100, window = 10, negative=5, hs=0, min_count=2, sample = 0, alpha=0.025, min_alpha=0.001, dm_mean = 0, dbow_words=1)
    model = model_doc2vec(model_d2v, preprocessed_tagged_dataset, 100)
    return model

def get_vectors(input_data, model):
    preprocessed_tagged_input_data= pre_processing_data(input_data)
    id, vectors = np.array(vector_for_learning(model, preprocessed_tagged_input_data), dtype=object)
    return vectors

In [5]:
# Rocchio algorithm
def rocchio_algorithm(query_doc_vector, docs_relevant_vectors,
                      docs_irrelevant_vectors, key_relevant_vectors,
                      key_irrelevant_vectors,
                      alpha, beta, gamma, delta):

    # sum_of_rel_doc_vectors = docs_relevant_vectors.sum(axis=0)
    # print('sum_of_rel_doc_vectors', sum_of_rel_doc_vectors)
    sum_of_rel_doc_vectors=[]
    sum_of_irrel_doc_vectors=[]
    sum_of_rel_key_vectors=[]
    sum_of_irrel_key_vectors=[]
    for each_rel_doc_vector in docs_relevant_vectors:
        if len(sum_of_rel_doc_vectors) == 0:
            # print('Length of sum_of_rel_doc_vectors is zero')
            sum_of_rel_doc_vectors = each_rel_doc_vector
        else:
            # print(each_rel_doc_vector)
            sum_of_rel_doc_vectors = list(map(operator.add, sum_of_rel_doc_vectors, each_rel_doc_vector))
            # print('SUM :', sum_of_rel_doc_vectors)
    for each_irrel_doc_vector in docs_irrelevant_vectors:
        if len(sum_of_irrel_doc_vectors) == 0:
            sum_of_irrel_doc_vectors = each_irrel_doc_vector
        else:
            sum_of_irrel_doc_vectors = list(map(operator.add, sum_of_irrel_doc_vectors, each_irrel_doc_vector))
    # sum_of_irrel_doc_vectors = docs_irrelevant_vectors.sum(axis=0)
    # print('sum_of_irrel_doc_vectors', sum_of_irrel_doc_vectors)
    for each_rel_key_vector in key_relevant_vectors:
        if len(sum_of_rel_key_vectors) == 0:
            sum_of_rel_key_vectors = each_rel_key_vector
        else:
            sum_of_rel_key_vectors = list(map(operator.add, sum_of_rel_key_vectors, each_rel_key_vector))
    # sum_of_rel_key_vectors = key_relevant_vectors.sum(axis=0)
    # print('sum_of_rel_key_vectors', sum_of_rel_key_vectors)
    for each_irrel_key_vector in key_irrelevant_vectors:
        if len(sum_of_irrel_key_vectors) == 0:
            sum_of_irrel_key_vectors = each_irrel_key_vector
        else:
            sum_of_irrel_key_vectors = list(map(operator.add, sum_of_irrel_key_vectors, each_irrel_key_vector))
    # sum_of_irrel_key_vectors = key_irrelevant_vectors.sum(axis=0)
    # print('sum_of_irrel_key_vectors', (delta/len(docs_relevant_vectors)) * np.array(sum_of_irrel_key_vectors))

    new_doc_vector_query = np.sum([np.array(query_doc_vector)
    , (alpha/len(docs_relevant_vectors)) * np.array(sum_of_rel_doc_vectors)
    , (beta/len(docs_irrelevant_vectors)) * np.array(sum_of_irrel_doc_vectors)
    , (gamma/len(docs_relevant_vectors)) * np.array(sum_of_rel_key_vectors)
    , (delta/len(docs_relevant_vectors)) * np.array(sum_of_irrel_key_vectors)], axis=0)
    df = pd.DataFrame({"a": [new_doc_vector_query]})
    return df.values

# Compute cosine scores
def compute_cosine_sim(new_query_vector, all_data, name):
    consine_similarities = []
    for index, data in all_data.iterrows():
        cosine_sim = cosine_similarity([np.array(new_query_vector[0])], [np.array(all_data[name][index])])
        consine_similarities.append(cosine_sim[0][0])
    return consine_similarities

# Rank data
def rank_data(new_query_vector, dataset, name, a, b):
    ps = PorterStemmer()
    cosine_sim_values = compute_cosine_sim(new_query_vector, dataset, name)
    dataset['Cosine_Similarity_' + name] = cosine_sim_values
    # dataset.drop(dataset[dataset['bntStyle'] == False].index, inplace=True)
    dataset['contexts'] = dataset['contexts'].apply(ast.literal_eval)
    rel_keyList_df = dataset['KeyList'][dataset['relevant'] == True]
    relevant_KeyList = []
    for lst in rel_keyList_df:
        # lst = list(map(lambda word: ps.stem(word), lst))
        relevant_KeyList.extend(lst)

    # dataset['Key_word_match_count'] = 0
    dataset['Keyword_match_dict'] = [{} for _ in range(len(dataset))]
    dataset['Key_word_match_count'] = 0
    dataset['matched_contexts'] = [[] for _ in range(len(dataset))]
    for i, row in dataset.iterrows():
        matched_context = []
        key_match_dict = {phrase: 0 for phrase in relevant_KeyList}
        contexts = row['contexts']
        for context in contexts:
            # words = [ps.stem(w.lower()) for w in nltk.word_tokenize(context)]
            for phrase in relevant_KeyList:
                # word_list = ps.stem(word).split()
                # set1 = set(word_list)
                # set2 = set(words)
                # print("Set1",set1)
                # print("Set2",set2)
                # if set1.issubset(set2):
                if phrase.lower() in context.lower():
                    key_match_dict[phrase] += 1
                    dataset.at[i, 'Key_word_match_count']+=1
                    matched_context.append(context)
        dataset.at[i, 'Keyword_match_dict'] = key_match_dict
        if matched_context is not []:
            dataset.at[i,'matched_contexts'] = list(set(matched_context))
        else:
            dataset.at[i,'matched_contexts'] = matched_context
    # columns_to_search = ['context_key_1', 'context_key_2', 'context_key_3', 'context_key_4', 'context_key_5']
    # for i, row in dataset.iterrows():
    #     for col in dataset[columns_to_search].columns:
    #         if row[col] is None:
    #             continue
    #         words = [ps.stem(w.lower()) for w in nltk.word_tokenize(row[col])]
    #         for word in relevant_KeyList:
    #             word_list = word.split()
    #             set1 = set(word_list)
    #             set2 = set(words)
    #             if set1.intersection(set2):
    #                 dataset.at[i, 'Key_word_match_count'] += 1
    # dataset['feedback_score'] = dataset['Key_word_match_count']/5 + dataset['Cosine_Similarity_' + name]
    dataset['feedback_score'] = a * dataset['Cosine_Similarity_' + name] / np.linalg.norm(dataset['Cosine_Similarity_' + name]) + b * dataset['Key_word_match_count'] / np.linalg.norm(dataset['Key_word_match_count'])
    # dataset['feedback_score'] = dataset['Cosine_Similarity_' + name]
    # sorted_dataset = dataset.sort_values(by=['feedback_score'], ascending=False)
    sorted_dataset = dataset.sort_values(by=['feedback_score'], ascending=False)
    return sorted_dataset



In [2]:
monoT5 = MonoT5ReRanker(text_field='abstract')
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
cord19_df = pd.read_csv('cord19_contextList.csv')
# model = define_d2v_model(cord19_df['abstract'])
model = joblib.load('gensim_model_2.pkl')
query_df = pd.DataFrame
reranked_df = pd.DataFrame

def search_query(query):
  index_ref2 = pt.IndexRef.of(pt_index_path + "/data.properties")
  index2 = pt.IndexFactory.of(index_ref2)
  # print(query)
  if not pt.started():
      pt.init()
  br = pt.BatchRetrieve(index2) % 50
  pipeline = (br >> pt.text.get_text(dataset, 'abstract') >> monoT5)
  search_result = pipeline.search(query)
  # print(search_result)
  search_result = search_result.drop_duplicates(subset=['docno'])
  # search_result = search_result.drop_duplicates(subset=['summary'])

  filtered_docs = pd.merge(cord19_df, search_result, on = "docno", how = "inner")
  filtered_docs.rename(columns={'abstract_x':'abstract'}, inplace=True)
  filtered_docs.drop(columns='abstract_y', axis=1, inplace=True)
  filtered_docs['KeyList'] = filtered_docs['KeyList'].apply(ast.literal_eval)
  # filtered_docs.to_csv('search_result_covid.csv')
    # filtered_docs.to_csv('search_result_cough.csv')
    # filtered_docs.to_csv('search_result_pandemic.csv')
    # filtered_docs.to_csv('search_result_pandemicDeath.csv')
    # filtered_docs.to_csv('search_result_hosBedAvb.csv')
  # filtered_docs = filtered_docs.drop_duplicates(subset=['docno', 'summary'])
  # return filtered_docs
  return filtered_docs


# search_query('covid')

NameError: name 'MonoT5ReRanker' is not defined

In [7]:
def get_reranking(feedback_df : pd.DataFrame, a, b):
    global  query_df
    feedback_df["doc_vec"] = get_vectors(feedback_df['abstract'], model)
    feedback_df["keyList_vec"] = get_vectors(feedback_df['KeyList'], model)
    relevant_df = feedback_df.head(10).loc[feedback_df['relevant'] == True]
    irrelevant_df = feedback_df.head(10).loc[feedback_df['relevant'] == False]
    query_df["query_vec"]  = get_vectors(query_df['query'].values, model)
    query_df['new_query_vec'] = rocchio_algorithm(query_df["query_vec"][0], relevant_df["doc_vec"].values, irrelevant_df["doc_vec"].values, relevant_df["keyList_vec"].values, irrelevant_df["keyList_vec"].values, 0.8, 0.3,0.8, 0.3)
    sorted_dataset = rank_data(query_df['new_query_vec'].values, feedback_df, "keyList_vec", a, b)
    return sorted_dataset

# Based on PCA
def generate_plot():
    rel_vecs = []
    irrel_vecs = []
    new_reranked_rel_vecs = []
    # plt.canvas.flush_events()
    print("Reranked DF: ", reranked_df)
    for i, data in enumerate(reranked_df.loc[reranked_df['relevant'] == True]['keyList_vec'].values):
        rel_vecs.append(data)
    for i, data in enumerate(reranked_df.loc[reranked_df['bntStyle'] == False]['keyList_vec'].values):
        irrel_vecs.append(data)
    for i, data in enumerate(reranked_df.head(10).loc[reranked_df['relevant'] == False]['keyList_vec'].values):
        new_reranked_rel_vecs.append(data)
    pca = PCA(n_components=2)
    pca_rel = pca.fit_transform(rel_vecs)
    print("Irrelevant vecs: ",irrel_vecs)
    pca_irrel = pca.fit_transform(irrel_vecs)
    pca_new_rel = pca.fit_transform(new_reranked_rel_vecs)
    query =np.concatenate(([np.array(query_df.iloc[0]['query_vec'])], [np.array(query_df.iloc[0]['new_query_vec'])]), axis=0)
    pca_query = pca.fit_transform(query)
    fig = Figure()
    axis = fig.add_subplot(1, 1, 1)
    axis.scatter(pca_rel[:,0], pca_rel[:,1], marker="8", label="relevant")
    axis.scatter(pca_irrel[:,0], pca_irrel[:,1], marker="*", label="irrelevant")
    axis.scatter(pca_new_rel[:,0], pca_new_rel[:,1], color='yellow', marker="d", label="new relevant")
    axis.scatter(pca_query[0][0], pca_query[0][1], marker="s", label="old_query")
    axis.scatter(pca_query[1][0], pca_query[1][1], marker="s", label="new_query")
    axis.set_xlabel("PCA 1")
    axis.set_ylabel("PCA 2")
    axis.set_title("Visualization of vectors")
    axis.axvline(x=0, color='gray', linestyle='--')
    axis.legend()

    return fig

In [8]:
# Based on TSNE
def generate_plot2():
    rel_vecs = []
    irrel_vecs = []
    new_reranked_rel_vecs = []
    print("Reranked DF: ", reranked_df)
    for i, data in enumerate(reranked_df.loc[reranked_df['relevant'] == True]['keyList_vec'].values):
        rel_vecs.append(data)
    for i, data in enumerate(reranked_df.loc[reranked_df['bntStyle'] == False]['keyList_vec'].values):
        irrel_vecs.append(data)
    for i, data in enumerate(reranked_df.head(10).loc[reranked_df['relevant'] == False]['keyList_vec'].values):
        new_reranked_rel_vecs.append(data)
    all_vecs = np.concatenate((rel_vecs, irrel_vecs, new_reranked_rel_vecs), axis=0)
    pca = TSNE(n_components=2, random_state = 0)
    pca_all = pca.fit_transform(all_vecs)
    pca_rel = pca_all[:len(rel_vecs)]
    pca_irrel = pca_all[len(rel_vecs):len(rel_vecs)+len(irrel_vecs)]
    pca_new_rel = pca_all[len(rel_vecs)+len(irrel_vecs):]
    query =np.concatenate(([np.array(query_df.iloc[0]['query_vec'])], [np.array(query_df.iloc[0]['new_query_vec'])]), axis=0)
    pca_query = pca.fit_transform(query)
    fig = Figure()
    axis = fig.add_subplot(1, 1, 1)
    axis.scatter(pca_rel[:,0], pca_rel[:,1], marker="8", label="relevant")
    axis.scatter(pca_irrel[:,0], pca_irrel[:,1], marker="*", label="irrelevant")
    axis.scatter(pca_new_rel[:,0], pca_new_rel[:,1], color='yellow', marker="d", label="new relevant")
    axis.scatter(pca_query[0][0], pca_query[0][1], marker="s", label="old_query")
    axis.scatter(pca_query[1][0], pca_query[1][1], marker="s", label="new_query")
    axis.axhline(y=0, linestyle='--', color='gray')
    axis.axvline(x=0, linestyle='--', color='gray')
    axis.set_xlabel('PCA 1')
    axis.set_ylabel('PCA 2')
    axis.set_title('Visualization of vectors')
    axis.legend()

    return fig

In [ ]:
# Run once to train Gensim Model

# model = define_d2v_model(cord19_df[~cord19_df['abstract'].isna()]['abstract'])
#
# # Save the model as a pickle in a file
# joblib.dump(model, 'gensim_model_2.pkl')

In [9]:
# Load Gensim model everytime

model = joblib.load('gensim_model_2.pkl')


In [ ]:
app = flask.Flask(__name__)

CORS(app)
@app.route('/query', methods=['GET'])
def search():
    query = request.args.get('searchString')
    print(query)

    searchResults = search_query(query)
    searchResults['clean_summary'] = searchResults['summary'].str.lower().apply(lambda x: re.sub(r'[^\w\s]','',x.lower()))
    searchResults.drop_duplicates(subset=['clean_summary'], keep='first',inplace=True)
    searchResults.drop('clean_summary', axis=1, inplace=True)
    searchResults.drop_duplicates(subset=['title'], keep='first',inplace=True)
    searchResults.drop_duplicates(subset=['docno'], keep='first',inplace=True)
    searchResults.drop_duplicates(subset=['abstract'], keep='first',inplace=True)

    # searchResults.to_csv(query + '.csv')
    # searchResults = pd.read_csv('breathing issue.csv')
    # searchResults = pd.read_csv('search_result_covid.csv')
    # searchResults = pd.read_csv('search_result_pandemic.csv')
    # searchResults = pd.read_csv('search_result_hosBedAvb.csv')
    # searchResults = pd.read_csv('search_result_dryCough.csv')
    # searchResults = pd.read_csv('search_result_hosBedAvb.csv')

    searchResults.drop(columns=['Unnamed: 0'], inplace=True)
    searchResults['KeyList'] = searchResults['KeyList'].apply(ast.literal_eval)
    # end of temporary code

    global query_df
    query_dict = {'query': [query], 'query_vec' : [np.nan], 'new_query_vec' : [np.nan] }
    query_df = pd.DataFrame(data=query_dict, index=[0])
    return searchResults.to_json(orient='records')

# Feedback
@app.route('/feedback', methods=['POST'])
def fetchFeedback():
    global reranked_df
    feedbackJson = request.json['updates']
    relevanceList = []
    if len(feedbackJson):
        for doc in feedbackJson:
            for relevance in doc['value']:
                relevanceList.append(relevance)
    feedback_df = pd.DataFrame(relevanceList)
    print(feedback_df.head(10))

    reranked_df = get_reranking(feedback_df, 0.3, 0.7)
    feedback_df= reranked_df.drop(reranked_df[reranked_df['bntStyle'] == False].index)

    # feedback_df.to_csv(query_df['query'][0] + '_feedback.csv')
    # feedback_df = pd.read_csv('breathing issue_feedback.csv')
    # feedback_df.drop(columns=['Unnamed: 0'], inplace=True)

    return feedback_df.to_json(orient='records')


@app.route('/plot')
def plot():
    # Generate the plot
    fig = generate_plot2()
    canvas = FigureCanvas(fig)
    output = io.BytesIO()
    canvas.print_png(output)
    response = Response(output.getvalue(), mimetype='image/png')

    return response


if __name__ == "__main__":
    app.run(host='0.0.0.0',port=5280)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5280
 * Running on http://141.44.228.253:5280
Press CTRL+C to quit


breathing issue


127.0.0.1 - - [28/Jun/2023 08:52:36] "GET /query?searchString=breathing%20issue HTTP/1.1" 200 -
127.0.0.1 - - [28/Jun/2023 08:52:46] "OPTIONS /feedback HTTP/1.1" 200 -


   Unnamed: 0.1     docno                                              title  \
0           662  471hzpyf  Noninvasive positive pressure ventilation for ...   
1          6712  roeom0yv  “Breathing” Hemagglutinin Reveals Cryptic Epit...   
2          6739  s25jy7ur  A Chemical-Biological-Radio-Nuclear (CBRN) Fil...   
3         11429  4giwnrbw                 Sudden Death from Pulmonary Causes   
4         12264  wc3mw30l                              Respiratory Disorders   
5         15628  018lcuxh  Assessment of spontaneous breathing during pre...   
6         17849  vpaxhalf  Should Intermittent Mandatory Ventilation Be A...   
7         18888  zzuce33t  Smartphone-Based Self-Testing of COVID-19 Usin...   
8         19678  lioxz2ku  Breath analysis for detection of viral infecti...   
9         19925  1wosekco  Diaphragm function and weaning from mechanical...   

                                            abstract  \
0  Noninvasive positive pressure ventilation (NPP...   
1  A un

...

127.0.0.1 - - [28/Jun/2023 09:23:17] "OPTIONS /feedback HTTP/1.1" 200 -


   Unnamed: 0.1     docno                                              title  \
0           662  471hzpyf  Noninvasive positive pressure ventilation for ...   
1          6712  roeom0yv  “Breathing” Hemagglutinin Reveals Cryptic Epit...   
2          6739  s25jy7ur  A Chemical-Biological-Radio-Nuclear (CBRN) Fil...   
3         11429  4giwnrbw                 Sudden Death from Pulmonary Causes   
4         12264  wc3mw30l                              Respiratory Disorders   
5         15628  018lcuxh  Assessment of spontaneous breathing during pre...   
6         17849  vpaxhalf  Should Intermittent Mandatory Ventilation Be A...   
7         18888  zzuce33t  Smartphone-Based Self-Testing of COVID-19 Usin...   
8         19678  lioxz2ku  Breath analysis for detection of viral infecti...   
9         19925  1wosekco  Diaphragm function and weaning from mechanical...   

                                            abstract  \
0  Noninvasive positive pressure ventilation (NPP...   
1  A un

127.0.0.1 - - [28/Jun/2023 09:24:26] "POST /feedback HTTP/1.1" 200 -


Reranked DF:      Unnamed: 0.1     docno                                              title  \
0            662  471hzpyf  Noninvasive positive pressure ventilation for ...   
2           6739  s25jy7ur  A Chemical-Biological-Radio-Nuclear (CBRN) Fil...   
1           6712  roeom0yv  “Breathing” Hemagglutinin Reveals Cryptic Epit...   
14         34170  jsuuckg6  Humidification performance of humidifying devi...   
37         91210  hiwapvdj  Bloom Energy powers two California sites in fa...   
20         40736  8m8zp3zb  Patient-ventilator interaction during acute lu...   
17         37851  qq1u29sr  Real time noninvasive estimation of work of br...   
38         92709  dz10xq98  Review of Issues and Challenges of Practicing ...   
30         60805  8j8enwc5  Acute bronchiolitis: assessment and management...   
35         79169  ausfl6fy               Efficacy of slit lamp breath shields   
43        117340  0z10ahnz  Symmetric cutaneous vasculitis in COVID‐19 pne...   
29         584

127.0.0.1 - - [28/Jun/2023 09:25:34] "GET /plot HTTP/1.1" 200 -
